# Jupyter练习1：完整优化流程

## 📚 代码示例关联

**本示例位置**：这是网络优化的综合练习

**基于**：
- [从零实现优化算法](../代码示例/01_从零实现优化算法.ipynb) - 理解优化算法原理
- [从零实现前馈神经网络](../../01_神经网络基础/02_前馈神经网络/代码示例/01_从零实现前馈神经网络.ipynb) - 理解网络训练流程

**扩展为**：
- [PyTorch基础入门](../../04_PyTorch_TensorFlow/01_PyTorch基础/代码示例/01_PyTorch基础入门.ipynb) - 使用框架的优化器
- [超参数优化](../05_超参数优化/代码示例/) - 优化算法的超参数调优
- [综合实战项目](../../08_综合实战项目/) - 完整项目中的优化应用

**相关示例**：
- [实现SGD优化器](../练习题/基础练习/练习2_实现SGD优化器.ipynb) - 基础练习
- [实现Momentum优化器](../练习题/基础练习/练习3_实现Momentum优化器.ipynb) - 基础练习
- [实现Adam优化器](../练习题/进阶练习/练习1_实现Adam优化器.ipynb) - 进阶练习
- [批量大小影响分析](../练习题/基础练习/练习4_批量大小影响分析.ipynb) - 批量大小分析

---

## 学习目标

通过本练习，你将完整地实现一个优化流程，包括：
1. 定义优化问题
2. 实现优化算法
3. 训练和评估
4. 结果分析

## 任务说明

本练习是一个综合性的实践项目，需要你完成所有TODO部分。


In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt

# 设置中文字体，确保图表能正确显示中文
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子，确保结果可复现
np.random.seed(42)

print("环境准备完成！")
print("Python版本要求：3.7+")
print("主要依赖：numpy, matplotlib")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

np.random.seed(42)
print("环境准备完成！")


## 步骤1：定义优化问题

TODO: 定义一个优化问题（损失函数和梯度函数）


In [ ]:
# 定义损失函数 - 使用Rosenbrock函数（经典的优化测试函数）
def loss_function(params):
    """
    Rosenbrock函数: f(x,y) = (a-x)^2 + b*(y-x^2)^2
    全局最小值在 (a, a^2) 处，值为0
    
    支持向量化计算：params可以是单个数组或二维数组
    """
    a, b = 1, 100
    if params.ndim == 1:
        x, y = params[0], params[1]
        return (a - x)**2 + b * (y - x**2)**2
    else:
        # 向量化版本，用于等高线计算
        x, y = params[..., 0], params[..., 1]
        return (a - x)**2 + b * (y - x**2)**2

# 定义梯度函数
def compute_gradients(params):
    """
    计算Rosenbrock函数的梯度
    """
    a, b = 1, 100
    x, y = params[0], params[1]
    
    # 对x的偏导数
    df_dx = -2 * (a - x) - 4 * b * x * (y - x**2)
    # 对y的偏导数
    df_dy = 2 * b * (y - x**2)
    
    return np.array([df_dx, df_dy])


## 步骤2：实现优化算法

TODO: 实现至少一种优化算法（SGD、Momentum或Adam）


In [ ]:
# 实现优化算法类
class SGD:
    """随机梯度下降优化器"""
    def __init__(self, learning_rate=0.01):
        self.lr = learning_rate
    
    def update(self, params, grads):
        return params - self.lr * grads


class Momentum:
    """带动量的梯度下降优化器"""
    def __init__(self, learning_rate=0.01, momentum=0.9):
        self.lr = learning_rate
        self.momentum = momentum
        self.v = None
    
    def update(self, params, grads):
        if self.v is None:
            self.v = np.zeros_like(params)
        
        self.v = self.momentum * self.v - self.lr * grads
        return params + self.v


class Adam:
    """Adam优化器"""
    def __init__(self, learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None  # 一阶矩估计
        self.v = None  # 二阶矩估计
        self.t = 0     # 时间步
    
    def update(self, params, grads):
        if self.m is None:
            self.m = np.zeros_like(params)
            self.v = np.zeros_like(params)
        
        self.t += 1
        
        # 更新一阶和二阶矩估计
        self.m = self.beta1 * self.m + (1 - self.beta1) * grads
        self.v = self.beta2 * self.v + (1 - self.beta2) * grads**2
        
        # 偏差修正
        m_hat = self.m / (1 - self.beta1**self.t)
        v_hat = self.v / (1 - self.beta2**self.t)
        
        # 更新参数
        return params - self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)


## 步骤3：训练过程

TODO: 实现训练循环


In [ ]:
# 初始化参数 - 从远离最优解的位置开始
params_sgd = np.array([-1.5, 2.0])
params_momentum = np.array([-1.5, 2.0])
params_adam = np.array([-1.5, 2.0])

# 创建不同的优化器
optimizer_sgd = SGD(learning_rate=0.001)
optimizer_momentum = Momentum(learning_rate=0.001, momentum=0.9)
optimizer_adam = Adam(learning_rate=0.01, beta1=0.9, beta2=0.999)

# 训练参数
n_epochs = 1000

# 记录训练过程
losses_sgd = []
losses_momentum = []
losses_adam = []
params_history_sgd = [params_sgd.copy()]
params_history_momentum = [params_momentum.copy()]
params_history_adam = [params_adam.copy()]

# 实现训练循环 - SGD
print("开始训练 SGD...")
for epoch in range(n_epochs):
    # 计算梯度
    grads = compute_gradients(params_sgd)
    
    # 更新参数
    params_sgd = optimizer_sgd.update(params_sgd, grads)
    
    # 记录损失和参数
    loss = loss_function(params_sgd)
    losses_sgd.append(loss)
    params_history_sgd.append(params_sgd.copy())
    
    if (epoch + 1) % 200 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss:.6f}, Params: [{params_sgd[0]:.4f}, {params_sgd[1]:.4f}]")

# 实现训练循环 - Momentum
print("\n开始训练 Momentum...")
for epoch in range(n_epochs):
    grads = compute_gradients(params_momentum)
    params_momentum = optimizer_momentum.update(params_momentum, grads)
    loss = loss_function(params_momentum)
    losses_momentum.append(loss)
    params_history_momentum.append(params_momentum.copy())
    
    if (epoch + 1) % 200 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss:.6f}, Params: [{params_momentum[0]:.4f}, {params_momentum[1]:.4f}]")

# 实现训练循环 - Adam
print("\n开始训练 Adam...")
for epoch in range(n_epochs):
    grads = compute_gradients(params_adam)
    params_adam = optimizer_adam.update(params_adam, grads)
    loss = loss_function(params_adam)
    losses_adam.append(loss)
    params_history_adam.append(params_adam.copy())
    
    if (epoch + 1) % 200 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss:.6f}, Params: [{params_adam[0]:.4f}, {params_adam[1]:.4f}]")

print("\n训练完成！")


## 步骤4：结果分析和可视化

TODO: 可视化优化过程和结果


In [ ]:
# 绘制损失曲线
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(losses_sgd, label='SGD', alpha=0.7)
plt.plot(losses_momentum, label='Momentum', alpha=0.7)
plt.plot(losses_adam, label='Adam', alpha=0.7)
plt.xlabel('迭代次数')
plt.ylabel('损失值')
plt.title('损失曲线对比')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')  # 使用对数刻度以便更好地观察

# 可视化优化路径（二维问题）
plt.subplot(1, 2, 2)
# 创建等高线图（使用向量化计算提高效率）
x = np.linspace(-2, 2, 100)
y = np.linspace(-1, 3, 100)
X, Y = np.meshgrid(x, y)
# 向量化计算损失函数值
params_grid = np.stack([X, Y], axis=-1)
Z = loss_function(params_grid)

# 绘制等高线
contour = plt.contour(X, Y, Z, levels=20, alpha=0.5)
plt.clabel(contour, inline=True, fontsize=8)

# 绘制优化路径
params_history_sgd = np.array(params_history_sgd)
params_history_momentum = np.array(params_history_momentum)
params_history_adam = np.array(params_history_adam)

plt.plot(params_history_sgd[:, 0], params_history_sgd[:, 1], 'o-', 
         label='SGD路径', markersize=3, alpha=0.6, linewidth=1.5)
plt.plot(params_history_momentum[:, 0], params_history_momentum[:, 1], 's-', 
         label='Momentum路径', markersize=3, alpha=0.6, linewidth=1.5)
plt.plot(params_history_adam[:, 0], params_history_adam[:, 1], '^-', 
         label='Adam路径', markersize=3, alpha=0.6, linewidth=1.5)

# 标记起点和终点
plt.plot(params_history_sgd[0, 0], params_history_sgd[0, 1], 'ko', 
         markersize=10, label='起点')
plt.plot(1, 1, 'r*', markersize=15, label='最优解 (1, 1)')

plt.xlabel('x')
plt.ylabel('y')
plt.title('优化路径可视化')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')

plt.tight_layout()
plt.show()

# 分析结果
print("\n" + "="*60)
print("优化结果分析")
print("="*60)
print(f"\nSGD优化器:")
print(f"  最终参数: [{params_sgd[0]:.6f}, {params_sgd[1]:.6f}]")
print(f"  最终损失: {losses_sgd[-1]:.6f}")
print(f"  最优解: [1.0, 1.0], 最优损失: 0.0")
print(f"  误差: {np.linalg.norm(params_sgd - np.array([1.0, 1.0])):.6f}")

print(f"\nMomentum优化器:")
print(f"  最终参数: [{params_momentum[0]:.6f}, {params_momentum[1]:.6f}]")
print(f"  最终损失: {losses_momentum[-1]:.6f}")
print(f"  误差: {np.linalg.norm(params_momentum - np.array([1.0, 1.0])):.6f}")

print(f"\nAdam优化器:")
print(f"  最终参数: [{params_adam[0]:.6f}, {params_adam[1]:.6f}]")
print(f"  最终损失: {losses_adam[-1]:.6f}")
print(f"  误差: {np.linalg.norm(params_adam - np.array([1.0, 1.0])):.6f}")

print("\n" + "="*60)
print("性能对比:")
print("="*60)
print(f"  最快收敛: ", end="")
if min(losses_sgd[-1], losses_momentum[-1], losses_adam[-1]) == losses_adam[-1]:
    print("Adam")
elif min(losses_sgd[-1], losses_momentum[-1], losses_adam[-1]) == losses_momentum[-1]:
    print("Momentum")
else:
    print("SGD")


## 扩展：学习率调整策略

下面演示如何使用学习率衰减策略来改进优化过程


In [ ]:
# 实现带学习率衰减的SGD优化器
class SGDWithDecay:
    """带学习率衰减的随机梯度下降优化器"""
    def __init__(self, learning_rate=0.01, decay_rate=0.95, decay_steps=100):
        self.initial_lr = learning_rate
        self.lr = learning_rate
        self.decay_rate = decay_rate
        self.decay_steps = decay_steps
        self.step = 0
    
    def update(self, params, grads):
        # 每decay_steps步衰减一次学习率
        if self.step % self.decay_steps == 0 and self.step > 0:
            self.lr *= self.decay_rate
        self.step += 1
        return params - self.lr * grads
    
    def get_lr(self):
        return self.lr


# 使用学习率衰减策略重新训练
print("="*60)
print("扩展实验：学习率衰减策略")
print("="*60)

params_decay = np.array([-1.5, 2.0])
optimizer_decay = SGDWithDecay(learning_rate=0.01, decay_rate=0.95, decay_steps=100)

losses_decay = []
lr_history = []
params_history_decay = [params_decay.copy()]

print("\n开始训练 SGD with Learning Rate Decay...")
for epoch in range(n_epochs):
    grads = compute_gradients(params_decay)
    params_decay = optimizer_decay.update(params_decay, grads)
    loss = loss_function(params_decay)
    losses_decay.append(loss)
    lr_history.append(optimizer_decay.get_lr())
    params_history_decay.append(params_decay.copy())
    
    if (epoch + 1) % 200 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss:.6f}, LR: {optimizer_decay.get_lr():.6f}, "
              f"Params: [{params_decay[0]:.4f}, {params_decay[1]:.4f}]")

print("\n训练完成！")

# 对比固定学习率和衰减学习率
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(losses_sgd, label='SGD (固定LR)', alpha=0.7)
plt.plot(losses_decay, label='SGD (衰减LR)', alpha=0.7)
plt.xlabel('迭代次数')
plt.ylabel('损失值')
plt.title('学习率衰减效果对比')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

plt.subplot(1, 3, 2)
plt.plot(lr_history, label='学习率变化', color='orange')
plt.xlabel('迭代次数')
plt.ylabel('学习率')
plt.title('学习率衰减曲线')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
# 绘制优化路径对比
x = np.linspace(-2, 2, 100)
y = np.linspace(-1, 3, 100)
X, Y = np.meshgrid(x, y)
params_grid = np.stack([X, Y], axis=-1)
Z = loss_function(params_grid)

contour = plt.contour(X, Y, Z, levels=20, alpha=0.3)
params_history_sgd = np.array(params_history_sgd)
params_history_decay = np.array(params_history_decay)

plt.plot(params_history_sgd[:, 0], params_history_sgd[:, 1], 'o-', 
         label='SGD (固定LR)', markersize=2, alpha=0.6, linewidth=1)
plt.plot(params_history_decay[:, 0], params_history_decay[:, 1], 's-', 
         label='SGD (衰减LR)', markersize=2, alpha=0.6, linewidth=1)
plt.plot(params_history_sgd[0, 0], params_history_sgd[0, 1], 'ko', 
         markersize=8, label='起点')
plt.plot(1, 1, 'r*', markersize=12, label='最优解')

plt.xlabel('x')
plt.ylabel('y')
plt.title('优化路径对比')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')

plt.tight_layout()
plt.show()

# 结果对比
print("\n" + "="*60)
print("学习率衰减策略效果分析")
print("="*60)
print(f"\n固定学习率 SGD:")
print(f"  最终损失: {losses_sgd[-1]:.6f}")
print(f"  误差: {np.linalg.norm(params_sgd - np.array([1.0, 1.0])):.6f}")

print(f"\n学习率衰减 SGD:")
print(f"  最终损失: {losses_decay[-1]:.6f}")
print(f"  误差: {np.linalg.norm(params_decay - np.array([1.0, 1.0])):.6f}")
print(f"  最终学习率: {optimizer_decay.get_lr():.6f}")

improvement = ((losses_sgd[-1] - losses_decay[-1]) / losses_sgd[-1]) * 100
print(f"\n改进幅度: {improvement:.2f}%")


## 扩展：不同学习率的影响

下面通过实验来观察不同学习率对SGD优化过程的影响


In [ ]:
# 测试不同学习率的影响
print("="*60)
print("扩展实验：不同学习率的影响")
print("="*60)

learning_rates = [0.0001, 0.001, 0.01, 0.1]
n_epochs_lr = 500  # 使用较少的迭代次数以便观察差异

results = {}
for lr in learning_rates:
    print(f"\n测试学习率: {lr}")
    params = np.array([-1.5, 2.0])
    optimizer = SGD(learning_rate=lr)
    losses = []
    params_history = [params.copy()]
    
    for epoch in range(n_epochs_lr):
        grads = compute_gradients(params)
        params = optimizer.update(params, grads)
        loss = loss_function(params)
        losses.append(loss)
        params_history.append(params.copy())
    
    results[lr] = {
        'losses': losses,
        'params_history': np.array(params_history),
        'final_loss': losses[-1],
        'final_params': params,
        'error': np.linalg.norm(params - np.array([1.0, 1.0]))
    }
    print(f"  最终损失: {losses[-1]:.6f}")
    print(f"  最终参数: [{params[0]:.4f}, {params[1]:.4f}]")
    print(f"  误差: {results[lr]['error']:.6f}")

# 可视化不同学习率的效果
plt.figure(figsize=(16, 5))

# 1. 损失曲线对比
plt.subplot(1, 3, 1)
colors = ['blue', 'green', 'orange', 'red']
for i, lr in enumerate(learning_rates):
    plt.plot(results[lr]['losses'], label=f'LR={lr}', 
             color=colors[i], alpha=0.7, linewidth=2)
plt.xlabel('迭代次数')
plt.ylabel('损失值')
plt.title('不同学习率的损失曲线')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

# 2. 优化路径对比
plt.subplot(1, 3, 2)
# 绘制等高线
x = np.linspace(-2, 2, 100)
y = np.linspace(-1, 3, 100)
X, Y = np.meshgrid(x, y)
params_grid = np.stack([X, Y], axis=-1)
Z = loss_function(params_grid)
contour = plt.contour(X, Y, Z, levels=20, alpha=0.3)

# 绘制不同学习率的优化路径
markers = ['o', 's', '^', 'D']
for i, lr in enumerate(learning_rates):
    params_hist = results[lr]['params_history']
    # 只绘制每10个点，避免路径过于密集
    plt.plot(params_hist[::10, 0], params_hist[::10, 1], 
             marker=markers[i], markersize=4, alpha=0.6, 
             linewidth=1.5, label=f'LR={lr}')

plt.plot(-1.5, 2.0, 'ko', markersize=10, label='起点')
plt.plot(1, 1, 'r*', markersize=15, label='最优解')
plt.xlabel('x')
plt.ylabel('y')
plt.title('不同学习率的优化路径')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')

# 3. 收敛速度对比（达到某个损失阈值所需的迭代次数）
plt.subplot(1, 3, 3)
thresholds = [100, 10, 1, 0.1, 0.01]
convergence_data = {lr: [] for lr in learning_rates}

for threshold in thresholds:
    for lr in learning_rates:
        losses = results[lr]['losses']
        # 找到首次低于阈值的迭代次数
        converged = False
        for epoch, loss in enumerate(losses):
            if loss < threshold:
                convergence_data[lr].append(epoch)
                converged = True
                break
        if not converged:
            convergence_data[lr].append(n_epochs_lr)  # 未收敛

# 绘制收敛速度
x_pos = np.arange(len(thresholds))
width = 0.2
for i, lr in enumerate(learning_rates):
    plt.bar(x_pos + i*width, convergence_data[lr], width, 
            label=f'LR={lr}', alpha=0.7, color=colors[i])

plt.xlabel('损失阈值')
plt.ylabel('收敛迭代次数')
plt.title('不同学习率的收敛速度')
plt.xticks(x_pos + width*1.5, [f'{t}' for t in thresholds])
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.yscale('log')

plt.tight_layout()
plt.show()

# 分析结果
print("\n" + "="*60)
print("学习率影响分析")
print("="*60)
print(f"\n{'学习率':<10} {'最终损失':<15} {'误差':<15} {'收敛性':<15}")
print("-" * 60)
for lr in learning_rates:
    final_loss = results[lr]['final_loss']
    error = results[lr]['error']
    # 判断是否收敛（损失小于0.01且误差小于0.1）
    converged = "✓ 收敛" if final_loss < 0.01 and error < 0.1 else "✗ 未收敛"
    print(f"{lr:<10.4f} {final_loss:<15.6f} {error:<15.6f} {converged:<15}")

print("\n关键观察:")
print("1. 学习率过小：收敛速度慢，但可能更稳定")
print("2. 学习率适中：平衡收敛速度和稳定性")
print("3. 学习率过大：可能震荡或不收敛")


## 总结

完成本练习后，你应该能够：
1. 理解优化问题的定义
2. 实现优化算法
3. 使用优化算法训练模型
4. 分析和可视化优化结果

### 思考问题

1. 不同优化算法的优缺点是什么？
2. 学习率对优化过程有什么影响？
3. 如何选择合适的优化算法？

### 扩展任务

- ✅ 尝试实现多种优化算法并对比（已完成：SGD、Momentum、Adam）
- ✅ 测试不同学习率的影响（见下方实验）
- ✅ 实现学习率调整策略（已完成：学习率衰减）
